Modelimizi oluşturduk ve bir isimlendirme ile kaydettik. Şimdi örnek olarak elimize yeni hasta kayıtları gelmiş olsun ve biz bu hastaları kaydettiğimiz modeli kullanarak diyabet olup olmadıklarının tahminini yapmak istiyoruz.

Bu sebeple öncelikle hasta kayıtlarını getirmemiz gerekiyor. Örnek olarak yeni hasta kayıtlarının eski diabetes.csv olduğunu düşünelim ve onun üzerinden hareket edelim.

In [1]:
import joblib
import pandas as pd

In [2]:
df = pd.read_csv("../datasets/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Şimdi modelimizi çağıralım.

In [4]:
new_model = joblib.load("voting_clf.pkl")

C:\Users\btskd\anaconda3\envs\pipeline_project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Şimdi veri setimizden rastgele bir hasta seçelim ve daha sonra bu hastayı modele verelim.

In [5]:
random_user = df.sample(1, random_state=45)

In [6]:
new_model.predict(random_user)

C:\Users\btskd\anaconda3\envs\pipeline_project\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Age
- BloodPressure
- DiabetesPedigreeFunction
- Glucose
- Insulin
- ...
Feature names seen at fit time, yet now missing:
- AGE
- BLOODPRESSURE
- DIABETESPEDIGREEFUNCTION
- GLUCOSE
- INSULIN
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 9 features, but KNeighborsClassifier is expecting 16 features as input.

Görüldüğü üzere hata aldık. Bunun sebebi; burada kullanmış olduğumuz modeli kurarken, ana veri setimizde birçok değişiklik yaptık. Yeni featurelar oluşturduk, encode işlemleri normalizasyon işlemleri vs. yaptık. Fakat burada modele verdiğimiz örnek ana df'den alınmış bir örnek. 

Dolayısıyla elimize gelen yeni verilerin bulunduğu veri setinin eski (modeli kurduğumuz) veri seti ile aynı olması, dönüştürülmesi lazım. Dolayısıyla bu veri setini benzer şekilde diabetes_data_pred modülümüzden/fonksiyonumuzdan geçirmemiz lazım. Bunu nasıl gerçekleştirebiliriz?

In [7]:
from diabetes_pipeline import diabetes_data_prep

C:\Users\btskd\anaconda3\envs\pipeline_project\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Pipeline dosyası içerisinde veri hazırlama bölümünün fonksiyonunu direkt olarak buraya import ettik. Eğer bu şekilde yapıldığında import edilmez ise, direkt olarak pipeline dosyasındaki veri hazırlama fonksiyonlarının hepsini buraya kopyala yapıştır bir şekilde taşımamız gerekir.

Yeni hastalarımız geldi. Bu veriyi prediction işlemi için aynı scriptten geçirmek zorundayız. 

**Dikkat!** Prediction yerine farklı birçok kaynakta bu bölüm karşımıza *scoring* olarak da gelebilir.

In [8]:
X, y = diabetes_data_prep(df)

Observations: 768
Variables: 13
cat_cols: 5
num_cols: 8
cat_but_car: 0
num_but_cat: 1
Observations: 768
Variables: 17
cat_cols: 9
num_cols: 8
cat_but_car: 0
num_but_cat: 1


Şimdi random bir hasta seçimini X üzerinden yapabiliriz.

In [9]:
random_user = X.sample(1, random_state=45)

In [29]:
new_model = joblib.load("voting_clf.pkl")

In [12]:
new_model.predict(random_user)

array([1], dtype=int64)

Başka bir hastada deneyelim:

In [13]:
random_user = X.sample(1, random_state=50)

In [14]:
new_model.predict(random_user)

array([0], dtype=int64)

Anlaşılacağı üzere, bu modeli tekrar kullanmak istediğimizde modelin anladığı ve bildiği bir şema var, o şemaya uygun bir şekilde değişken seti göndermemiz lazım bundan dolayı, buradaki veri ön işleme işlemini tekrar ele aldık. 

In [18]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [34]:
df = pd.read_csv("../datasets/diabetes.csv")

In [36]:
df = df.drop("Outcome", axis=1)

In [37]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [33]:
from diabetes_pipeline_copy import diabetes_data_prep

In [38]:
X = diabetes_data_prep(df)

Observations: 768
Variables: 12
cat_cols: 4
num_cols: 8
cat_but_car: 0
num_but_cat: 0
Observations: 768
Variables: 16
cat_cols: 8
num_cols: 8
cat_but_car: 0
num_but_cat: 0


In [39]:
X

,PREGNANCIES,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,AGE,NEW_GLUCOSE_CAT_PREDIABETES,NEW_AGE_CAT_OLD,NEW_AGE_CAT_YOUNG,NEW_BMI_RANGE_HEALTY,NEW_BMI_RANGE_OVERWEIGHT,NEW_BMI_RANGE_OBESE,NEW_BLOODPRESSURE_HS1,NEW_BLOODPRESSURE_HS2
0,0.639947,0.848324,0.149641,0.907270,-0.787602,0.204013,0.468492,1.425995,1,0,0,0,0,1,0,0
1,-0.844885,-1.123396,-0.160546,0.530902,-0.787602,-0.684422,-0.365061,-0.190672,0,0,1,0,1,0,0,0
2,1.233880,1.943724,-0.263941,-1.288212,-0.787602,-1.103255,0.604397,-0.105584,1,0,1,1,0,0,0,0
3,-0.844885,-0.998208,-0.160546,0.154533,0.217583,-0.494043,-0.920763,-1.041549,0,0,1,0,1,0,0,0
4,-1.141852,0.504055,-1.504687,0.907270,1.008900,1.409746,5.484909,-0.020496,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,1.137221,0.115169,-0.908682,2.532136,0,1,0,0,0,1,0,0
764,-0.547919,0.034598,0.046245,0.405445,-0.787602,0.610154,-0.398282,-0.531023,0,0,1,0,0,1,0,0
765,0.342981,0.003301,0.149641,0.154533,0.410066,-0.735190,-0.685193,-0.275760,0,0,1,0,1,0,0,0
766,-0.844885,0.159787,-0.470732,-1.288212,-0.787602,-0.240205,-0.371101,1.170732,0,0,0,0,0,1,0,0


In [40]:
random_user = X.sample(1, random_state=45)

In [41]:
new_model.predict(random_user)

array([1], dtype=int64)

In [31]:
random_user = X.sample(1, random_state=50)